In [1]:
from pdp_utils import *
import random
import numpy as np
import functools
import time
n = 1000

np.random.seed(20)

def resolve_max_capacity(prob, vehicle_idx):
    capacities = prob["VesselCapacity"]
    try:
        return capacities[vehicle_idx]
    # Index out of bounds: give dummy truck infinite capacity
    except:
        return float('inf')
    
def resolve_allowed_calls(prob, vehicle_idx):
    allowed_calls_list = prob["VesselCargo"]
    try:
        return allowed_calls_list[vehicle_idx]
    # Index out of bounds, allow dummy truck all calls
    except:
        return np.array([1. for i in range(prob["n_calls"])])

    


In [2]:
class Truck:
    
    def __init__(self, idx, max_capacity, allowed_calls):
        self.idx = idx
        self.max_capacity = max_capacity
        self.allowed_calls = allowed_calls
        self.actions = []
        self.active = True
        
    def take_call(self, call_idx, prob):
        self.actions.append(call_idx)
        pass
    
    def resolve_complete_call_time(self, call_idx, prob):
        return 0
    
    def resolve_complete_call_cost(self, call_idx, prob):
        return 0
    
        

In [3]:
def generate_trucks(prob):
    trucks = []
    for i in range(prob["n_vehicles"]):
        initial_call = np.random.choice(np.size(prob["Cargo"], 0))
        max_capacity = resolve_max_capacity(prob, i)
        allowed_calls = resolve_allowed_calls(prob, i)    
        truck = Truck(i, max_capacity, allowed_calls)
        trucks.append(truck)
    dummy_truck = Truck(prob["n_vehicles"], float('inf'), np.array([1 for i in range(len(prob["VesselCargo"][0]))]))
    return trucks, dummy_truck

In [4]:
def run_problem(problem):
    prob_take_call = 0.5
    prob_retire_truck = 0.1
    prob = load_problem(problem)
    cargos = prob["Cargo"].copy()
    trucks, dummy_truck = generate_trucks(prob)
    available_trucks = trucks.copy()
    
    # Distribute calls between trucks
    while len(cargos) > 0:
        to_select = np.random.choice(np.size(cargos, 0))
        current_call = cargos[to_select]
        call_idx = np.where(np.all(prob["Cargo"] == current_call,axis=1))[0][0]
        
        # Assign selected call to random truck, with prob_take_call probability of rejecting it.
        available_trucks = [truck for truck in available_trucks if truck.active]
        take_call_value = random.random()
        if(len(available_trucks) > 0 and take_call_value < prob_take_call):
            # Some truck can make the call: Select random truck among available trucks.
            truck = random.choice(available_trucks)
            truck.take_call(call_idx, prob)
        else:
            # Do not take call - give call to dummy truck, increment cost of rejected calls
            dummy_truck.take_call(call_idx, prob)
            dummy_truck.take_call(call_idx, prob)

        # Remove call from list of calls
        cargos = np.delete(cargos, to_select, axis=0)
        
        # Give trucks chance to accept no more calls with probability prob_retire_truck
        for truck in available_trucks:
            retire_truck_value = random.random()
            if(retire_truck_value < prob_retire_truck):
                truck.active = False
    
    # Let trucks decide in which order calls should be resolved
    for truck in (trucks):
        calls = truck.actions.copy()
        for call in calls:
            c_idx = truck.actions.index(call) + 1
            insert_idx = random.randint(c_idx, len(truck.actions))
            truck.actions.insert(insert_idx, call)

        
    solution = functools.reduce(lambda acc, val: acc + [i + 1 for i in val.actions] + [0], trucks, [])
    solution = solution + [i+1 for i in dummy_truck.actions]
    feasiblity, c = feasibility_check(solution, prob)
    sol_cost = cost_function(solution, prob)
    return solution, sol_cost, feasiblity, c, prob
    
    

## C7V3

In [5]:


start_time = time.time()

c7v3_init_sol, c7v3_init_cost, feasibility, c, init_prob = [], float('inf'), False, '', None
c7v3_best_sol = c7v3_init_sol
c7v3_best_cost = c7v3_init_cost
c7v3_total_cost = c7v3_init_cost
# Force start with feasible solution
while(not feasibility):
    c7v3_init_sol, c7v3_init_cost, feasibility, c, init_prob = run_problem('./Call_7_Vehicle_3.txt')
    c7v3_best_sol = c7v3_init_sol
    c7v3_best_cost = c7v3_init_cost
    c7v3_total_cost = c7v3_init_cost
feasibles = 1
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_7_Vehicle_3.txt')
    if(feasibility):
        feasibles += 1
        c7v3_total_cost += sol_cost
        if(sol_cost < c7v3_best_cost):
            c7v3_best_sol = solution
            c7v3_best_cost = sol_cost
end_time = time.time()

c7v3_avg_cost = c7v3_total_cost / feasibles
print(c7v3_best_sol)
print('init_cost', c7v3_init_cost)
print('best_cost', c7v3_best_cost)
print('avg_cost', c7v3_avg_cost)
print('improvement', ((c7v3_init_cost - c7v3_best_cost) / c7v3_init_cost)*100)
print('elapsed time', end_time - start_time)


[0, 3, 3, 0, 5, 5, 7, 7, 0, 6, 6, 1, 1, 4, 4, 2, 2]
init_cost 2405234.0
best_cost 2043429.0
avg_cost 2736946.0853658537
improvement 15.042403358675289
elapsed time 20.113099098205566


## C18V5

In [6]:

start_time = time.time()
c18v5_init_sol, c18v5_init_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
c18v5_init_cost = cost_function(c18v5_init_sol, prob)
c18v5_best_sol = c18v5_init_sol
c18v5_best_cost = c18v5_init_cost
c18v5_total_cost = c18v5_init_cost

# Force start with feasible solution
while(not feasibility):
    c18v5_init_sol, c18v5_init_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
    c18v5_init_cost = cost_function(c18v5_init_sol, prob)
    c18v5_best_sol = c18v5_init_sol
    c18v5_best_cost = c18v5_init_cost
    c18v5_total_cost = c18v5_init_cost
feasibles = 1
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
    if feasibility:
        feasibles += 1
        c18v5_total_cost += sol_cost
        if(sol_cost < c18v5_best_cost):
            c18v5_best_sol = solution
            c18v5_best_cost = sol_cost
end_time = time.time()

c18v5_avg_cost = c18v5_total_cost / feasibles
print(c18v5_best_sol)
print('init_cost', c18v5_init_cost)
print('best_cost', c18v5_best_cost)
print('avg_cost', c18v5_avg_cost)
print('improvement', ((c18v5_init_cost - c18v5_best_cost) / c18v5_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 9, 8, 9, 8, 13, 13, 0, 6, 6, 0, 5, 5, 0, 0, 12, 12, 7, 7, 10, 10, 3, 3, 14, 14, 15, 15, 16, 16, 11, 11, 4, 4, 1, 1, 17, 17, 18, 18, 2, 2]
init_cost 8402678.0
best_cost 6981644.0
avg_cost 8037526.545454546
improvement 16.911679823979927
elapsed time 35.530109882354736


## C35V7 

In [7]:
start_time = time.time()
c35v7_init_sol, c35v7_init_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
c35v7_init_cost = cost_function(c35v7_init_sol, prob)
c35v7_best_sol = c35v7_init_sol
c35v7_best_cost = c35v7_init_cost
c35v7_total_cost = c35v7_init_cost

# Force start with feasible solution
while not feasibility:
    c35v7_init_sol, c35v7_init_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
    c35v7_init_cost = cost_function(c35v7_init_sol, prob)
    c35v7_best_sol = c35v7_init_sol
    c35v7_best_cost = c35v7_init_cost
    c35v7_total_cost = c35v7_init_cost
feasibles = 1
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
    if feasibility:
        feasibles += 1
        c35v7_total_cost += sol_cost
        if(sol_cost < c35v7_best_cost):
            c35v7_best_sol = solution
            c35v7_best_cost = sol_cost
end_time = time.time()

c35v7_avg_cost = c35v7_total_cost / feasibles
print(c35v7_best_sol)
print('init_cost', c35v7_init_cost)
print('best_cost', c35v7_best_cost)
print('avg_cost', c35v7_avg_cost)
print('improvement', ((c35v7_init_cost - c35v7_best_cost) / c35v7_init_cost)*100)
print('elapsed time', end_time - start_time)

[21, 21, 0, 0, 0, 3, 3, 0, 5, 5, 2, 2, 0, 11, 11, 31, 31, 0, 0, 10, 10, 30, 30, 24, 24, 23, 23, 25, 25, 15, 15, 28, 28, 33, 33, 26, 26, 34, 34, 35, 35, 7, 7, 8, 8, 16, 16, 29, 29, 12, 12, 18, 18, 14, 14, 32, 32, 20, 20, 9, 9, 6, 6, 27, 27, 13, 13, 19, 19, 22, 22, 17, 17, 4, 4, 1, 1]
init_cost 17568709.0
best_cost 16185612.0
avg_cost 17115489.625
improvement 7.872502185561841
elapsed time 60.39966607093811


## C80V20

In [8]:
start_time = time.time()
c80v20_init_sol, c80v20_init_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
c80v20_init_cost = cost_function(c80v20_init_sol, prob)
c80v20_best_sol = c80v20_init_sol
c80v20_best_cost = c80v20_init_cost
c80v20_total_cost = c80v20_init_cost
while not feasibility:
    c80v20_init_sol, c80v20_init_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
    c80v20_init_cost = cost_function(c80v20_init_sol, prob)
    c80v20_best_sol = c80v20_init_sol
    c80v20_best_cost = c80v20_init_cost
    c80v20_total_cost = c80v20_init_cost
feasibles = 1
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
    if feasibility:
        feasibles += 1
        c80v20_total_cost += sol_cost
        if(sol_cost < c80v20_best_cost):
            c80v20_best_sol = solution
            c80v20_best_cost = sol_cost
end_time = time.time()

c80v20_avg_cost = c80v20_total_cost / feasibles
print(c80v20_best_sol)
print('init_cost', c80v20_init_cost)
print('best_cost', c80v20_best_cost)
print('avg_cost', c80v20_avg_cost)
print('improvement', ((c80v20_init_cost - c80v20_best_cost) / c80v20_init_cost)*100)
print('elapsed time', end_time - start_time)

[71, 71, 0, 0, 0, 1, 1, 0, 0, 49, 49, 0, 0, 0, 0, 18, 18, 0, 0, 0, 0, 0, 0, 0, 76, 76, 73, 73, 0, 0, 0, 0, 6, 6, 14, 14, 61, 61, 15, 15, 68, 68, 54, 54, 66, 66, 51, 51, 74, 74, 35, 35, 44, 44, 17, 17, 5, 5, 29, 29, 19, 19, 9, 9, 52, 52, 53, 53, 56, 56, 63, 63, 20, 20, 32, 32, 27, 27, 16, 16, 13, 13, 46, 46, 23, 23, 41, 41, 25, 25, 34, 34, 3, 3, 21, 21, 79, 79, 30, 30, 65, 65, 31, 31, 64, 64, 24, 24, 48, 48, 75, 75, 70, 70, 78, 78, 57, 57, 47, 47, 40, 40, 62, 62, 67, 67, 38, 38, 80, 80, 36, 36, 72, 72, 33, 33, 69, 69, 2, 2, 58, 58, 55, 55, 77, 77, 50, 50, 12, 12, 10, 10, 8, 8, 11, 11, 37, 37, 26, 26, 7, 7, 45, 45, 22, 22, 39, 39, 43, 43, 60, 60, 42, 42, 59, 59, 4, 4, 28, 28]
init_cost 45653233.0
best_cost 43904827.0
avg_cost 44606107.5
improvement 3.829752867666568
elapsed time 212.82745790481567


## C130V40

In [9]:
start_time = time.time()
c130v40_init_sol, c130v40_init_cost, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
c130v40_init_cost = cost_function(c130v40_init_sol, prob)
c130v40_best_sol = c130v40_init_sol
c130v40_best_cost = c130v40_init_cost
c130v40_total_cost = c130v40_init_cost
while not feasibility:
    c130v40_init_sol, c130v40_init_cost, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
    c130v40_init_cost = cost_function(c130v40_init_sol, prob)
    c130v40_best_sol = c130v40_init_sol
    c130v40_best_cost = c130v40_init_cost
    c130v40_total_cost = c130v40_init_cost
feasibles = 1
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
    if feasibility:
        feasibles += 1
        c130v40_total_cost += sol_cost
        if(sol_cost < c130v40_best_cost):
            c130v40_best_sol = solution
            c130v40_best_cost = sol_cost
end_time = time.time()

c130v40_avg_cost = c130v40_total_cost / feasibles
print(c130v40_best_sol)
print('init_cost', c130v40_init_cost)
print('best_cost', c130v40_best_cost)
print('avg_cost', c130v40_avg_cost)
print('improvement', ((c130v40_init_cost - c130v40_best_cost) / c130v40_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 0, 0, 0, 0, 56, 56, 0, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 92, 92, 0, 111, 111, 0, 109, 109, 0, 0, 82, 82, 0, 97, 97, 0, 0, 0, 74, 74, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 120, 0, 45, 45, 0, 0, 0, 0, 0, 39, 39, 13, 13, 1, 1, 128, 128, 101, 101, 55, 55, 14, 14, 34, 34, 93, 93, 7, 7, 5, 5, 90, 90, 30, 30, 95, 95, 62, 62, 73, 73, 29, 29, 80, 80, 31, 31, 64, 64, 54, 54, 125, 125, 123, 123, 59, 59, 94, 94, 52, 52, 48, 48, 89, 89, 126, 126, 41, 41, 10, 10, 4, 4, 60, 60, 76, 76, 35, 35, 67, 67, 112, 112, 28, 28, 40, 40, 96, 96, 32, 32, 114, 114, 53, 53, 127, 127, 103, 103, 43, 43, 110, 110, 2, 2, 75, 75, 17, 17, 118, 118, 19, 19, 83, 83, 100, 100, 11, 11, 130, 130, 81, 81, 46, 46, 42, 42, 71, 71, 106, 106, 107, 107, 129, 129, 119, 119, 61, 61, 37, 37, 8, 8, 102, 102, 77, 77, 25, 25, 91, 91, 68, 68, 65, 65, 66, 66, 47, 47, 50, 50, 6, 6, 122, 122, 26, 26, 78, 78, 38, 38, 22, 22, 88, 88, 36, 36, 18, 18, 104, 104, 124, 124, 57, 57, 116, 116, 108, 108, 86, 86, 115, 115, 99, 99, 49, 49, 70, 70, 5

## C300V90

In [ ]:
from tqdm import tqdm

start_time = time.time()
c300v90_init_sol, c300v90_init_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
c300v90_best_sol = c300v90_init_sol
c300v90_best_cost = c300v90_init_cost
c300v90_total_cost = c300v90_init_cost
ctr = 0
while not feasibility:
    ctr += 1
    if ctr % 10 == 0:
        print(ctr)
    c300v90_init_sol, c300v90_init_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
    c300v90_best_sol = c300v90_init_sol
    c300v90_best_cost = c300v90_init_cost
    c300v90_total_cost = c300v90_init_cost
feasibles = 1
for i in tqdm(range(n)):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
    if feasibility:
        feasibles += 1
        c300v90_total_cost += sol_cost
        if(sol_cost < c300v90_best_cost):
            c300v90_best_sol = solution
            c300v90_best_cost = sol_cost
end_time = time.time()

c300v90_avg_cost = c300v90_total_cost / feasibles
print(c300v90_best_sol)
print('init_cost', c300v90_init_cost)
print('best_cost', c300v90_best_cost)
print('avg_cost', c300v90_avg_cost)
print('improvement', ((c300v90_init_cost - c300v90_best_cost) / c300v90_init_cost)*100)
print('elapsed time', end_time - start_time)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222